In [2]:
from sqlalchemy import create_engine
import pandas as pd

### Create Database

Open pgAdmin and right-click on your postgres server (mine's postgres 10 bc my Mac refused to work with 11 and 12), but for most of you, it should be 12. Then go to "create" and "database."

<img src="./dbInstructionImages/createDB.png" width="60%">

Then name your database "aq_db" and save.

<img src="./dbInstructionImages/saveDB.png" width="40%">

In [3]:
# Create connection string to database. Note that my port number is 5433.
# Once again this is due to my Mac not playing nice. Most likely, your port number is 5432 so make sure to change.
# Additionally, your username and password are most likely postgres:postgres as they're specified here.

rds_connection_string = "postgres:postgres@localhost:5433/aq_db"

In [4]:
# Set up your database engine

engine = create_engine(f'postgresql://{rds_connection_string}')

In [6]:
# Read our csv into a dataframe using pandas

aq_df = pd.read_csv('./Shane/output_data/AirNowAPI_20200411_073607.csv')
aq_df.head(20)

,DateObserved,HourObserved,LocalTimeZone,ReportingArea,StateCode,Latitude,Longitude,ParameterName,AQI,AQICategoryNumber,AQICategory,cityRank,cityName,cityCSA,cityState,cityPopulation2018Estimate,cityPopulation2010Census,cityLat,cityLong,cityZip
0,1/1/15,0,EST,Newark,NJ,40.7267,-74.1442,OZONE,30,1,Good,1,New York City,"New York-Newark, NY-NJ-CT-PA Combined Statisti...",NY,22679948,22255491,40.7128,-74.006,10001
1,1/1/15,0,EST,Newark,NJ,40.7267,-74.1442,PM2.5,65,2,Moderate,1,New York City,"New York-Newark, NY-NJ-CT-PA Combined Statisti...",NY,22679948,22255491,40.7128,-74.006,10001
2,1/2/15,0,EST,Newark,NJ,40.7267,-74.1442,OZONE,24,1,Good,1,New York City,"New York-Newark, NY-NJ-CT-PA Combined Statisti...",NY,22679948,22255491,40.7128,-74.006,10001
3,1/2/15,0,EST,Newark,NJ,40.7267,-74.1442,PM2.5,56,2,Moderate,1,New York City,"New York-Newark, NY-NJ-CT-PA Combined Statisti...",NY,22679948,22255491,40.7128,-74.006,10001
4,1/3/15,0,EST,Newark,NJ,40.7267,-74.1442,OZONE,8,1,Good,1,New York City,"New York-Newark, NY-NJ-CT-PA Combined Statisti...",NY,22679948,22255491,40.7128,-74.006,10001
5,1/3/15,0,EST,Newark,NJ,40.7267,-74.1442,PM2.5,53,2,Moderate,1,New York City,"New York-Newark, NY-NJ-CT-PA Combined Statisti...",NY,22679948,22255491,40.7128,-74.006,10001
6,1/4/15,0,EST,Newark,NJ,40.7267,-74.1442,OZONE,20,1,Good,1,New York City,"New York-Newark, NY-NJ-CT-PA Combined Statisti...",NY,22679948,22255491,40.7128,-74.006,10001
7,1/4/15,0,EST,Newark,NJ,40.7267,-74.1442,PM2.5,37,1,Good,1,New York City,"New York-Newark, NY-NJ-CT-PA Combined Statisti...",NY,22679948,22255491,40.7128,-74.006,10001
8,1/5/15,0,EST,Newark,NJ,40.7267,-74.1442,OZONE,31,1,Good,1,New York City,"New York-Newark, NY-NJ-CT-PA Combined Statisti...",NY,22679948,22255491,40.7128,-74.006,10001
9,1/5/15,0,EST,Newark,NJ,40.7267,-74.1442,PM2.5,31,1,Good,1,New York City,"New York-Newark, NY-NJ-CT-PA Combined Statisti...",NY,22679948,22255491,40.7128,-74.006,10001


In [22]:
aq_df['Datetime'] = pd.to_datetime(aq_df['DateObserved'], format="%m/%d/%y")

# creates a third date column which captures only the YYYY value of the date from the new 'Datetime' column 
aq_df['year'] = aq_df['Datetime'].dt.strftime("%Y")

# creates a fourth date column which captures only the full month name value of the date from the new 'Datetime' column (i.e. 'January', 'February', etc)
aq_df['month'] = aq_df['Datetime'].dt.strftime("%B")

# filters data by user selected date and cityname 
selected_data = aq_df.loc[(aq_df["year"] == '2015'), :]  

# Isolates Ozone data points for the selected year and city 
ozone_data = selected_data.loc[selected_data["ParameterName"] == 'OZONE', :]

# Groups the ozone data by month and AQI Quality (good, moderate, etc.), then gets a count of each quality category for each month
#grouped_ozone_data = ozone_data.groupby(['month', 'AQICategory'])
#count_ozone = grouped_ozone_data['AQICategory'].count()

ozone_data.groupby('cityName')['AQI'].mean().to_dict()

{'Atlanta': 45.69135802469136,
 'Boston': 27.63095238095238,
 'Charlotte': 41.476190476190474,
 'Chicago': 45.023809523809526,
 'Dallas': 54.142857142857146,
 'Denver': 52.833333333333336,
 'Detroit': 41.23809523809524,
 'Houston-The Woodlands, TX Combined Statistical Area': 56.464285714285715,
 'Los Angeles': 51.642857142857146,
 'Miami': 32.13095238095238,
 'Minneapolis': 31.754716981132077,
 'New York City': 36.38095238095238,
 'Orlando': 33.96103896103896,
 'Philadelphia': 46.17857142857143,
 'Phoenix': 56.154761904761905,
 'Pittsburgh': 46.345238095238095,
 'Portland': 30.19277108433735,
 'Sacramento': 35.28048780487805,
 'Salt Lake City': 44.23809523809524,
 'San Antonio': 44.45238095238095,
 'San Francisco': 26.89156626506024,
 'Seattle': 25.038961038961038,
 'St. Louis': 40.903614457831324,
 'Washington': 46.23809523809524}

In [29]:
# Use the dataframe's 'to_sql' method to create a new table named 'city_aq', using the engine we created earlier.
# The first two lines ensure that we drop the table if it already exists so we can create our table using
# our dataframe method. (For example, if our csv data changes and we need to update the table.)

with engine.connect() as connection:
    connection.execute("DROP TABLE IF EXISTS city_aq")

df.to_sql('city_aq',con=engine,index=False)

If you return to pgAdmin, you'll see our newly created table in the database. (If you don't, you may need to right-click on 'Tables' and refresh.)

<img src="./dbInstructionImages/table.png" width="40%">

In [31]:
# Here you can see our table was properly created as we're reading it back into pandas

df_fromPG = pd.read_sql('city_aq',con=engine)
df_fromPG.head()

,DateObserved,HourObserved,LocalTimeZone,ReportingArea,StateCode,Latitude,Longitude,ParameterName,AQI,AQICategoryNumber,AQICategory,cityRank,cityName,cityCSA,cityState,cityPopulation2018Estimate,cityPopulation2010Census,cityLat,cityLong,cityZip
0,1/1/15,0,EST,Newark,NJ,40.7267,-74.1442,OZONE,30,1,Good,1,New York City,"New York-Newark, NY-NJ-CT-PA Combined Statisti...",NY,22679948,22255491,40.7128,-74.006,10001
1,1/1/15,0,EST,Newark,NJ,40.7267,-74.1442,PM2.5,65,2,Moderate,1,New York City,"New York-Newark, NY-NJ-CT-PA Combined Statisti...",NY,22679948,22255491,40.7128,-74.006,10001
2,1/2/15,0,EST,Newark,NJ,40.7267,-74.1442,OZONE,24,1,Good,1,New York City,"New York-Newark, NY-NJ-CT-PA Combined Statisti...",NY,22679948,22255491,40.7128,-74.006,10001
3,1/2/15,0,EST,Newark,NJ,40.7267,-74.1442,PM2.5,56,2,Moderate,1,New York City,"New York-Newark, NY-NJ-CT-PA Combined Statisti...",NY,22679948,22255491,40.7128,-74.006,10001
4,1/3/15,0,EST,Newark,NJ,40.7267,-74.1442,OZONE,8,1,Good,1,New York City,"New York-Newark, NY-NJ-CT-PA Combined Statisti...",NY,22679948,22255491,40.7128,-74.006,10001


Additionally, if you open up the query tool in pgAdmin (right click on aq_db in left column and click 'Query Tool'), you can execute an explicit SQL command to show the table data.

<img src='./dbInstructionImages/querytool.png' width='60%'>

#### With this table now set-up, python-flask can connect to the database, read in the data, and filter it according to our front-end inputs.

In [7]:
cities = aq_df['cityName'].unique()
cities

array(['New York City', 'Los Angeles', 'Chicago', 'Washington',
       'San Francisco', 'Boston', 'Dallas', 'Philadelphia',
       'Houston-The Woodlands, TX Combined Statistical Area', 'Miami',
       'Atlanta', 'Detroit', 'Phoenix', 'Seattle', 'Orlando',
       'Minneapolis', 'Denver', 'Portland', 'St. Louis', 'Charlotte',
       'Sacramento', 'Pittsburgh', 'Salt Lake City', 'San Antonio'],
      dtype=object)

In [8]:
lats = aq_df['Latitude'].unique()
lats

array([40.7267, 34.0663, 41.964 , 38.919 , 37.75  , 42.351 , 32.767 ,
       39.95  , 29.751 , 25.776 , 33.65  , 42.233 , 33.543 , 47.562 ,
       28.505 , 44.955 , 39.768 , 45.538 , 38.75  , 35.227 , 38.567 ,
       40.434 , 40.777 , 29.417 ])

In [9]:
lons = aq_df['Longitude'].unique()
lons

array([ -74.1442, -118.2266,  -87.659 ,  -77.013 , -122.43  ,  -71.051 ,
        -96.783 ,  -75.151 ,  -95.351 ,  -80.211 ,  -84.43  ,  -83.333 ,
       -112.071 , -122.3405,  -81.374 ,  -93.185 , -104.873 , -122.656 ,
        -90.383 ,  -80.843 , -121.467 ,  -79.984 , -111.93  ,  -98.484 ])

In [10]:
cityDict = {'objects':[]}

In [12]:
for index,city in enumerate(cities):
    objectDict = {"circle":{"coordinates":[lats[index],lons[index]]},"city":city}
    cityDict['objects'].append(objectDict)

In [14]:
cityDict

{'objects': [{'circle': {'coordinates': [40.7267, -74.1442]},
   'city': 'New York City'},
  {'circle': {'coordinates': [34.0663, -118.2266]}, 'city': 'Los Angeles'},
  {'circle': {'coordinates': [41.964, -87.65899999999999]}, 'city': 'Chicago'},
  {'circle': {'coordinates': [38.919000000000004, -77.013]},
   'city': 'Washington'},
  {'circle': {'coordinates': [37.75, -122.43]}, 'city': 'San Francisco'},
  {'circle': {'coordinates': [42.351000000000006, -71.051]}, 'city': 'Boston'},
  {'circle': {'coordinates': [32.766999999999996, -96.78299999999999]},
   'city': 'Dallas'},
  {'circle': {'coordinates': [39.95, -75.15100000000001]},
   'city': 'Philadelphia'},
  {'circle': {'coordinates': [29.750999999999998, -95.351]},
   'city': 'Houston-The Woodlands, TX Combined Statistical Area'},
  {'circle': {'coordinates': [25.776, -80.211]}, 'city': 'Miami'},
  {'circle': {'coordinates': [33.65, -84.43]}, 'city': 'Atlanta'},
  {'circle': {'coordinates': [42.233000000000004, -83.333]},
   'city

In [15]:
import json

In [16]:
prepJson = json.dumps(cityDict)
prepJson

'{"objects": [{"circle": {"coordinates": [40.7267, -74.1442]}, "city": "New York City"}, {"circle": {"coordinates": [34.0663, -118.2266]}, "city": "Los Angeles"}, {"circle": {"coordinates": [41.964, -87.65899999999999]}, "city": "Chicago"}, {"circle": {"coordinates": [38.919000000000004, -77.013]}, "city": "Washington"}, {"circle": {"coordinates": [37.75, -122.43]}, "city": "San Francisco"}, {"circle": {"coordinates": [42.351000000000006, -71.051]}, "city": "Boston"}, {"circle": {"coordinates": [32.766999999999996, -96.78299999999999]}, "city": "Dallas"}, {"circle": {"coordinates": [39.95, -75.15100000000001]}, "city": "Philadelphia"}, {"circle": {"coordinates": [29.750999999999998, -95.351]}, "city": "Houston-The Woodlands, TX Combined Statistical Area"}, {"circle": {"coordinates": [25.776, -80.211]}, "city": "Miami"}, {"circle": {"coordinates": [33.65, -84.43]}, "city": "Atlanta"}, {"circle": {"coordinates": [42.233000000000004, -83.333]}, "city": "Detroit"}, {"circle": {"coordinates

In [17]:
with open('circles.json', 'w') as outfile:
    json.dump(cityDict, outfile)